In [15]:
import dotenv
dotenv.load_dotenv()
import tensorflow as tf
import pandas as pd
from rouge_score import rouge_scorer
import evaluate
import os
import csv

# Evaluation

## ROUGE F1 Scores

An older-fashioned statistical set of metrics for similarity, typically used for summarizations.

In [18]:
rouge_calcer = rouge_scorer.RougeScorer([
    'rougeL'
], use_stemmer = True)

def calc_rouge(short_text, full_text):
    try:
        rouges = rouge_calcer.score(short_text, full_text)
        return rouges['rougeL'][2]
    except Exception as e:
        return 0.0

def calc_rouge_pair(row):
    return calc_rouge(row['description'], row['transcription']), calc_rouge(row[f'model-summary'], row['transcription'])

## BERTScore

Uses embeddings from the BERT transformer model to judge similarity.

In [20]:
bertscore = evaluate.load("bertscore")

def calc_bertscore(short_text, full_text):
    try:
        result = bertscore.compute(
                predictions = [short_text],
                references = [full_text],
                model_type = "microsoft/deberta-large-mnli",
                lang = "en",
                device = "cuda:1"
        )

        return result["f1"][0]
    except:
        return 0.0

def calc_bertscore_pair(row):
    return pd.Series([calc_bertscore(row['description'], row['transcription']), calc_bertscore(row[f'model-summary'], row['transcription'])])

## BLEURT

A regression model based on BERT.

In [22]:
bleurt = evaluate.load("bleurt", config_name = "bleurt-tiny-128")

def calc_bleu(short_text, full_text):
    # this was slower on GPU, I think it doesn't paralleize well because of how small the model is
    with tf.device("/CPU:0"):
        try:
            result = bleurt.compute(
                    predictions = [short_text],
                    references = [full_text],
                    )
            return result["scores"][0]
        except:
            return 0.0

def calc_bleu_pair(row):
    return pd.Series([calc_bleu(row['description'], row['transcription']), calc_bleu(row[f'model-summary'], row['transcription'])])

INFO:tensorflow:Reading checkpoint /mnt/data/hf/metrics/bleurt/bleurt-tiny-128/downloads/extracted/599cd3ff6a3bbad54e145d867ccea405bb98c2b832fb29b50fb02089a1026530/bleurt-tiny-128.


INFO:tensorflow:Reading checkpoint /mnt/data/hf/metrics/bleurt/bleurt-tiny-128/downloads/extracted/599cd3ff6a3bbad54e145d867ccea405bb98c2b832fb29b50fb02089a1026530/bleurt-tiny-128.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Will load checkpoint bert_custom


...

INFO:tensorflow:Creating Eager Mode predictor.


INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


## HHEM 2.1 model from Vectara (WIP)

Languaged-model-based system for detecting hallucinations in other LM operations.

In [24]:
os.environ["WANDB_BASE_URL"] = "https://api.wandb.ai"
from weave.scorers import WeaveHallucinationScorerV1

hallucination_scorer = WeaveHallucinationScorerV1(device = 'cuda:1')

def hallucination_score(query, context, output):
    try:
        result = hallucination_scorer.score(
            query = query,
            context = context,
            output = output,            
        )
    
        return result.metadata['score']
    except:
        return 1.0

wandb: Downloading large artifact hallucination_hhem_scorer:v0, 421.31MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:0.9 (461.7MB/s)
Device set to use cuda:1


## Coherence

A fine-tuned deberta-small-long-nli Small Language Model that ensures the writing doesn't contradict itself. 

In [26]:
import tensorflow as tf
os.environ["USE_TF"] = "0"
os.environ["WANDB_BASE_URL"] = "https://api.wandb.ai"
from weave.scorers import WeaveCoherenceScorerV1

coherence_scorer = WeaveCoherenceScorerV1(device = 'cuda:1')

def coherence_score(query, output):
    try:
        result = coherence_scorer.score(
            query = query,
            output = output
        )
    
        return result.metadata['score']
    except:
        return 0.0

wandb: Downloading large artifact coherence_scorer:v0, 549.59MB. 21 files... 
wandb:   21 of 21 files downloaded.  
Done. 0:0:1.2 (448.2MB/s)
Device set to use cuda:1


## Calculate and Save

In [16]:
model_names = [
    'biobart',
    'gemma',
    'llama',    
]

for test_model_name in model_names:
    df = pd.read_csv(f'./data/mtsamples_with_{test_model_name}.csv')
    df['transcription'] = df.transcription.astype(str)

    df[['rougeL_f1_source', 'rougeL_f1_dest']] = df.apply(calc_rouge_pair, axis = 1, result_type = 'expand')
    df[['bertscore_f1_source', 'bertscore_f1_dest']] = df.apply(calc_bertscore_pair, axis = 1)
    df[['bleurt_source', 'bleurt_dest']] = df.apply(calc_bleu_pair, axis = 1)
    df['hallucination-score'] = df.apply(lambda x: hallucination_score(x.transcription, x.description, x['model-summary']), axis = 1)
    df['coherence-score_source'] = df.apply(lambda x: coherence_score(x.transcription, x['description']), axis = 1)
    df['coherence-score_dest'] = df.apply(lambda x: coherence_score(x.transcription, x['model-summary']), axis = 1)

    # Save this model's stats to disk
    df.to_csv(f'./data/mtsamples_with_{test_model_name}_model_scores.csv', index = False, quoting = csv.QUOTE_NONNUMERIC)

---

# Aggregate

## Load the metrics for all models

In [ ]:
df = pd.read_csv('./data/mtsamples_with_biobart_model_scores.csv')
df = pd.concat([df, pd.read_csv('./data/mtsamples_with_gemma_model_scores.csv')], ignore_index = True)
df = pd.concat([df, pd.read_csv('./data/mtsamples_with_llama_model_scores.csv')], ignore_index = True)

## Data exploration

In [ ]:
df.head()

In [ ]:
df.groupby('model-name').count().iloc[:, 0]

# Metrics for the human-entered summaries

the source values are the same for all models, so just pick one and get the averages

In [ ]:
df_metrics_human =  df[df['model-name'] == 'biobart'][['rougeL_f1_source', 'bertscore_f1_source', 'bleurt_source', 'coherence-score_source']].mean()
df_metrics_human

In [ ]:
# Metrics for the machine-generated summaries for each model

In [ ]:
df_metrics_models = df.groupby('model-name')[['rougeL_f1_dest', 'bertscore_f1_dest', 'bleurt_dest', 'hallucination-score', 'coherence-score_dest']].mean()
df_metrics_models

In [ ]:
# Comparison of machine-generated and human-generated

In [ ]:
pd.concat([
        df_metrics_models.rougeL_f1_dest - df_metrics_human.rougeL_f1_source, 
        df_metrics_models.bertscore_f1_dest - df_metrics_human.bertscore_f1_source,
        df_metrics_models.bleurt_dest - df_metrics_human.bleurt_source,
        df_metrics_models['coherence-score_dest'] - df_metrics_human['coherence-score_source'],
    
    ], axis = 1).rename(columns = {'rougeL_f1_dest': 'rougeL_f1_diff', 
                                   'bertscore_f1_dest': 'bertscore_f1_diff', 
                                   'bleurt_dest': 'bleurt_diff', 
                                   'coherence-score_dest': 'coherence-score_diff'})
    